In [2]:
import numpy as np
from pyqubo import Binary
import pyqubo
import neal
import matplotlib.pyplot as plt 
from sympy import *
from datetime import datetime
import pandas as pd


## Input Data 

In [3]:
#input data 


#Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].to_numpy()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].to_numpy()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].to_numpy()

n_assets = len(asset_quantity)

#Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].to_numpy()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].to_numpy()

n_accounts = len(account_exposure)

#single limits
single_limits = pd.read_csv("data/sample_single_limits.csv", header=None).to_numpy() 

#haircuts 
haircuts = pd.read_csv("data/sample_haircuts.csv", header=None).to_numpy()/100

#Updated Tier list 
cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])

# Penalty Terms

In [4]:

#Penalty Terms
lambda_cost_fn = 1e3
lambda_consistency = 1
lambda_exposure = 1



# Defining the QUBO

In [5]:
# Binary variables

n_binary = 7
max_binary_value = 2**(n_binary) - 1 

binary_coeff_vector = np.zeros(n_binary)

for i in range(n_binary):
    binary_coeff_vector[i] = 2**i


#Decision variable

binary_variables = []

for i in range(n_assets):
    for j in range(n_accounts):
        for k in range(n_binary):
            globals()["x" + str(i + 1) + str(j + 1) + str(k+1)] = Binary("x" + str(i + 1) + str(j + 1) + str(k+1))
            binary_variables.append(globals()["x" + str(i + 1) + str(j + 1) + str(k + 1)])


#Slack variable for consistency 
num_slack_consistency_arr = np.zeros(shape=(n_assets))

for i in range(n_assets):
    num_slack_consistency_arr[i] = int(np.ceil(np.log2(max_binary_value)))


consistency_slack_variables_arr = [] 

for count, i in enumerate(num_slack_consistency_arr): 
    consistency_slack_variables_arr.append(pyqubo.Array.create("consistency_asset" + str(count + 1) + "_slack", shape = int(i), vartype = "BINARY"))

#single limits
single_limits_fractional = np.zeros_like(single_limits) 
single_limits_num_bits = np.zeros((n_assets,n_accounts), dtype=int) 


for i in range(n_assets):
    for j in range(n_accounts):
        single_limits_fractional[i][j] = single_limits[i][j]/asset_quantity[i]
        aux_variable = int(np.floor(np.log2(single_limits_fractional[i][j]*max_binary_value)))
        single_limits_num_bits[i][j] = n_binary if aux_variable > n_binary else aux_variable
    



# Constructing the QUBO equation 

This involves the QUBO objective function, the consistency contraints, the exposure constraints.

In [6]:

#QUBO objective function 

cost_fn = 0

for i in range(n_assets):
    for j in range(n_accounts):
        aux_term1 = 0
        for k in range(single_limits_num_bits[i][j]):
            aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
        cost_fn += aux_term1*cost_factor_matrix[i][j]

cost_fn /= max_binary_value

#for normalisation and scaling
cost_fn*=1000 

In [7]:
# Consistency constraint 

consistency_penalty_term = 0

#slack part
consistency_penalty_slack = []

for i in consistency_slack_variables_arr:
    aux_term = 0
    for exponent,j in enumerate(i):
        aux_term += (2**exponent)*j
    consistency_penalty_slack.append(aux_term)


#decision variable part
for i in range(n_assets):
    aux_term2 = 0
    for j in range(n_accounts):
        aux_term1 = 0
        for k in range(single_limits_num_bits[i][j]):
            aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
        aux_term2 += (aux_term1)
    consistency_penalty_term += (aux_term2 - max_binary_value + consistency_penalty_slack[i])**2

#Normalising
consistency_penalty_term /= (2**((n_binary - 1)*2))
#Scaling
consistency_penalty_term *= 10000

In [8]:
#Exposure constraint

exposure_penalty_term = 0

for j in range(n_accounts):
    aux_term2 = 0
    for i in range(n_assets):
        aux_term1 = 0
        for k in range(single_limits_num_bits[i][j]):
            aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
        aux_term2 += (aux_term1*asset_quantity[i]*asset_value[i]*haircuts[i][j])/max_binary_value
    exposure_penalty_term += (aux_term2 - account_exposure[j]) **2 

#normalising
exposure_penalty_term /= ((max(asset_quantity*asset_value)*64/max_binary_value))**2
#scaling
exposure_penalty_term *= 1e8



In [9]:


#Final QUBO
QUBO_equation =lambda_cost_fn*cost_fn + lambda_consistency*consistency_penalty_term  + lambda_exposure*exposure_penalty_term 

#compile model
model = QUBO_equation.compile()
qubo, offset = model.to_qubo()


# Run Solver

In [10]:
#Run Solver

bqm = model.to_bqm()
sa = neal.SimulatedAnnealingSampler()
num_reads = 100
num_sweeps = 1000
sampleset = sa.sample(bqm, num_reads=num_reads, num_sweeps=num_sweeps)

#Get best sample
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
results = best_sample.sample
pprint(f"The minimum energy is: {best_sample.energy}")


The minimum energy is: 646948.1852855533


# Verify Results

In [11]:

exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage=====")

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        decimal = 0
        for k in range(single_limits_num_bits[i][j]):
            decimal +=binary_coeff_vector[k]*results.get("x" + str(i + 1) + str(j + 1) + str(k+1))
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]
    allocation_solution[i] = asset


=====Each Individual Allocation Percentage=====
Assign 5.51% of asset 1 to account 1
Assign 9.45% of asset 1 to account 2
Assign 0.00% of asset 1 to account 3
Assign 0.00% of asset 1 to account 4
Assign 0.00% of asset 1 to account 5
Assign 37.01% of asset 2 to account 1
Assign 11.81% of asset 2 to account 2
Assign 2.36% of asset 2 to account 3
Assign 0.00% of asset 2 to account 4
Assign 1.57% of asset 2 to account 5
Assign 0.00% of asset 3 to account 1
Assign 0.00% of asset 3 to account 2
Assign 0.00% of asset 3 to account 3
Assign 0.00% of asset 3 to account 4
Assign 0.00% of asset 3 to account 5
Assign 37.01% of asset 4 to account 1
Assign 11.81% of asset 4 to account 2
Assign 0.00% of asset 4 to account 3
Assign 0.00% of asset 4 to account 4
Assign 0.00% of asset 4 to account 5
Assign 3.94% of asset 5 to account 1
Assign 11.81% of asset 5 to account 2
Assign 18.11% of asset 5 to account 3
Assign 1.57% of asset 5 to account 4
Assign 2.36% of asset 5 to account 5
Assign 0.00% of asset

In [12]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")  

====Validation for Consistency====
14.96% of asset 1 posted
52.76% of asset 2 posted
0.00% of asset 3 posted
48.82% of asset 4 posted
37.80% of asset 5 posted
7.09% of asset 6 posted
0.00% of asset 7 posted
56.69% of asset 8 posted
8.66% of asset 9 posted
0.00% of asset 10 posted


In [13]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i]:,} required") 
    print(f"{(exposure_solution[i]/account_exposure[i] - 1) *100.:0.2f}%")

=====Validation for Exposure======
$815,162.7112894212 of collateral posted, $824,000.0 required
-1.07%
$663,815.8496806776 of collateral posted, $669,292.0 required
-0.82%
$536,463.8232428432 of collateral posted, $548,600.0 required
-2.21%
$68,706.82793528514 of collateral posted, $64,844.52 required
5.96%
$465,692.72465245664 of collateral posted, $472,000.0 required
-1.34%


In [15]:
print("===Validation of Single limits =====")
for i in range(n_assets):
    for j in range(n_accounts):
        decimal = 0
        for k in range(single_limits_num_bits[i][j]):
            decimal +=(2**k)*results.get("x" + str(i + 1) + str(j + 1) + str(k+1))
        allocation = (decimal/max_binary_value)*asset_quantity[i] 
        print(f"Amount allocated: {allocation:0.2f}. Single Limit: {single_limits[i][j]:0.2f}")
        print(f"Constraint Satisfied?: {allocation <= single_limits[i][j]}")
        

===Validation of Single limits =====
Amount allocated: 10536.69. Single Limit: 20997.72
Constraint Satisfied?: True
Amount allocated: 18062.90. Single Limit: 153498.10
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 27959.58
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 4957.23
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 12027.82
Constraint Satisfied?: True
Amount allocated: 83447.20. Single Limit: 225485.00
Constraint Satisfied?: True
Amount allocated: 26632.09. Single Limit: 47932.09
Constraint Satisfied?: True
Amount allocated: 5326.42. Single Limit: 8730.80
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 5159.90
Constraint Satisfied?: True
Amount allocated: 3550.94. Single Limit: 225485.00
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 634872.00
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 789538.89
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 1

# Export to csv

In [15]:
#Saving to an array
Q_value_balanced_py_qubo = np.zeros(shape=(n_assets,n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        decimal = 0
        for k in range(single_limits_num_bits[i][j]):
            decimal +=binary_coeff_vector[k]*results.get("x" + str(i + 1) + str(j + 1) + str(k+1))
        Q_value_balanced_py_qubo[i][j] = decimal/max_binary_value

pprint(f"Objective value: {sum(sum(Q_value_balanced_py_qubo*cost_factor_matrix))}")

Objective value: 0.6354330708661418


In [ ]:
#Uncomment line below to save file to csv

#pd.DataFrame(Q_value_balanced_py_qubo).to_csv("Q_value_balanced_py_qubo_no_single_lim1.csv", header=None, index=None)
